In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv(os.path.join('.', 'colombia_data_sample.csv'))
df = df.astype({'dpto':'category', 
                'mpio':'category', 
                'cabecera':'bool',
                'cod_encuestas':'float64',
                'cant_vivienda':'float64',
                'cant_hog':'float64',
                'cant_per':'float64',
                'sexo':'category', 
                'edad':'float64',
                'parentesco':'category', 
                'lug_nac':'category', 
                'vivia_5anos':'category', 
                'vivia_1ano':'category', 
                'enfermo':'category', 
                'atendido':'bool', 
                'calidad_serv':'float64',
                'discapacidad':'bool', 
                'alfabeta':'bool', 
                'nivel_educ':'category', 
                'trabajo':'category', 
                'est_civil':'category', 
                'hnvh':'float64', 
                'hnvm':'float64', 
                'hnvm':'float64', 
                'hvh':'float64', 
                'hvm':'float64', 
                'hfch':'float64', 
                'hfcm':'float64'})
df['date_uhnv'] = pd.to_datetime(df['date_uhnv'], errors='coerce')
df.head()

,Unnamed: 0,dpto,mpio,cabecera,cod_encuestas,cant_vivienda,cant_hog,cant_per,sexo,edad,...,nivel_educ,trabajo,est_civil,hnvh,hnvm,hvh,hvm,hfch,hfcm,date_uhnv
0,0,5,45,False,32.0,1.0,1.0,1.0,Hombre,2.0,...,Basico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1,88,1,True,35.0,1.0,1.0,1.0,Hombre,4.0,...,Basico,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2,5,45,True,84.0,1.0,1.0,1.0,Mujer,6.0,...,Medio,Remunerado,Emparejado,2.0,0.0,2.0,0.0,NaN,NaN,2012-12-01
3,3,5,45,True,85.0,1.0,1.0,1.0,Hombre,8.0,...,Basico,Remunerado,Emparejado,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,4,5,45,True,97.0,1.0,1.0,1.0,Hombre,7.0,...,Medio,No_Remunerado,Emparejado,NaN,NaN,NaN,NaN,NaN,NaN,NaT


### Columnas:
1. **dpto**: Departamento. (string)
2. **mpio**: Municipalidad. (string)
3. **cabecera**: ¿Municipalidad cabecera? (boolean)
4. **cod_encuestas**: Código único de encuesta. (numeric)
5. **cant_vivienda**: Cantidad de viviendas por código de encuesta. (numeric)
6. **cant_hog**: Cantidad de hogares por vivienda. (numeric)
7. **cant_per**: Cantidad de personas por hogar. (numeric)
8. **sexo**: Sexo. (category: Hombre, Mujer)
9. **edad**: Edad. (numeric: rangos de 5 años, edad 1 comprende de 0 a 4 años cumplidos y 21 de 100 en adelante)
10. **parentesco**: Parentesco respecto a jefe de hogar. (category: Jefe_Hogar, Pareja, Hijo, Otro_Pariente, Otro_No_Pariente)
11. **lug_nac**: Nacimiento respecto a municipio actual. (category: Lugar_Actual, Lugar_Otro)
12. **vivia_5anos**: Vivienda respecto a municipio actual hace 5 años. (category: Lugar_Actual, Lugar_Otro)
13. **vivia_1ano**: Vivienda respecto a municipio actual hace 1 año. (category: Lugar_Actual, Lugar_Otro)
14. **enfermo**: Ayuda tomada la última vez que estuvo enfermo. NULL no ha estado enfermo en el último mes. (category: Profesional, No_Profesional, Nada)
15. **atendido**: ¿Fue atendido por la IPS? (boolean)
16. **calidad_serv**: Calificación del servicio de la IPS. (numeric)
17. **discapacidad**: ¿Persona discapacitada? (boolean)
18. **alfabeta**: ¿Es alfabeta? (bool)
19. **nivel_educ**: Nivel de educación máximo alcanzado. (category: Basico, Medio, Avanzado, Ninguno)
20. **trabajo**: Estado laboral. (category: Remunerado, No_Remunerado, Estudio, No_Apto)
21. **est_civil**: Estado civil. (category: Emparejado, Separado, Viudo, Soltero)
22. **hnvh**: Cantidad de hijos nacidos vivos hombres. (numeric)
23. **hnvm**: Cantidad de hijas nacidas vivas mujeres. (numeric)
24. **hvh**: Cantidad de hijos vivos hombres. (numeric)
25. **hvm**: Cantidad de hijas vivas mujeres. (numeric)
26. **hfch**: Cantidad de hijos fuera de Colombia hombres. (numeric)
27. **hfcm**: Cantidad de hijas fuera de Colombia mujeres. (numeric)
28. **date_uhnv**: Fecha de último hijo nacido vivo. (date)

## Idea principal
Se busca tomar una infografía base e irla filtrando hasta el punto que teniendo en cuenta las mismas variables se denotan diferencias de sexo en la cantidad de personas.

## Pre
Se asume que a lo largo de las generaciones Colombia será como una foto

## Escena 1
Municipios se dividen en clusters de manera que se confunda la variable socio-económica.

In [3]:
# Aquí irían los clusters, se dividieron entre cabeceras y no cabeceras en este caso
df['cabecera'].unique()

array([False,  True])

## Escena 2
Nacen N parejas de niños (M/F) en municipios escogidos dentro de los clusters. Análisis de edad 1 y 2 (0 a 9) en columna "enfermo", se espera que el valor sea distinto a "Nada". (Edad 1 y 2 no poseen información de trabajo)
¿Se debería filtrar por atención no recibida durante la enfermedad?
Análisis de columna "calidad_serv" buscando si existe diferencia entre sexo

In [4]:
for i, cabecera in enumerate(df['cabecera'].unique()):
    print('Para el cluster {0:d}'.format(i))
    # EDA
    df_slice = df[(df['edad'].isin([1, 2])) & (df['cabecera'] == cabecera)].copy()
    enfermo_counts_h = df_slice[df_slice['sexo'] == 'Hombre']['enfermo'].value_counts()
    enfermo_counts_m = df_slice[df_slice['sexo'] == 'Mujer']['enfermo'].value_counts()
    porcentaje_no_atend_h = enfermo_counts_h['Nada'] / enfermo_counts_h.sum()
    porcentaje_no_atend_m = enfermo_counts_m['Nada'] / enfermo_counts_m.sum()
    print('La cantidad de niños edad 1 que no reciben atención médica es {0:.2f}%'.format(porcentaje_no_atend_h * 100))
    print('La cantidad de niñas edad 1 que no reciben atención médica es {0:.2f}%'.format(porcentaje_no_atend_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['enfermo'] == 'Nada')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Atención médica {0:s}es independiente de sexo'.format('no ' if p_val < 0.05 else ''))
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['calidad_serv'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Calidad de atención {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de niños edad 1 que no reciben atención médica es 2.16%
La cantidad de niñas edad 1 que no reciben atención médica es 1.98%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo

Para el cluster 1
La cantidad de niños edad 1 que no reciben atención médica es 2.13%
La cantidad de niñas edad 1 que no reciben atención médica es 1.81%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo



## Escena 3: 
Edad 3 (10 a 14) se analiza columna "trabajo", se espera tenga el valor de "Estudio". Se filtra población.

In [5]:
for i, cabecera in enumerate(df['cabecera'].unique()):
    print('Para el cluster {0:d}'.format(i))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cabecera'] == cabecera)].copy()
    estudio_counts_h = df_slice[df_slice['sexo'] == 'Hombre']['trabajo'].value_counts()
    estudio_counts_m = df_slice[df_slice['sexo'] == 'Mujer']['trabajo'].value_counts()
    porcentaje_no_estud_h = 1 - (estudio_counts_h['Estudio'] / estudio_counts_h.sum())
    porcentaje_no_estud_m = 1 - (estudio_counts_m['Estudio'] / estudio_counts_m.sum())
    print('La cantidad de jovenes hombres edad 3 que no estudian es {0:.2f}%'.format(porcentaje_no_estud_h * 100))
    print('La cantidad de jovenes mujeres edad 3 que no estudian es {0:.2f}%'.format(porcentaje_no_estud_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['trabajo'] == 'Estudio')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de jovenes hombres edad 3 que no estudian es 6.81%
La cantidad de jovenes mujeres edad 3 que no estudian es 6.64%
Estudio es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres edad 3 que no estudian es 2.49%
La cantidad de jovenes mujeres edad 3 que no estudian es 2.64%
Estudio no es independiente de sexo



## Escena 4:
Edad 3 (10 a 14) se espera que no haya relación entre los hijos que tienen y su actividad académica, se tiene en cuenta cuadro de contingencia entre hijos nacidos ("hnvh" + "hnvm") respecto a "trabajo", si es significativo se filtra población.

In [6]:
for i, cabecera in enumerate(df['cabecera'].unique()):
    print('Para el cluster {0:d}'.format(i))
    # Slicing
    df_slice = df[(df['edad'] == 3) & (df['cabecera'] == cabecera)].copy()
    df_slice['hnvt'] = df_slice['hnvh'].fillna(0.0) + df_slice['hnvm'].fillna(0.0)
    max_digit = len(str(max(df_slice.groupby('hnvt')['hnvt'].count())))
    for ind, grouped in df_slice.groupby('hnvt'):
        # EDA
        estudio_counts = grouped['trabajo'].value_counts()
        porcentaje_no_estud = 1 - (estudio_counts['Estudio'] / estudio_counts.sum())
        print('En edad 3, {0:<{3:.0f}.0f} jovenes tienen {1:.0f} hijo(s), de los cuales el {2:>6.2f}% abandonan sus estudios' \
              .format(grouped['hnvt'].count(), ind, porcentaje_no_estud * 100, max_digit))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['hnvt'], columns = df_slice['trabajo'] == 'Estudio')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de cantidad de hijos\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
En edad 3, 9946 jovenes tienen 0 hijo(s), de los cuales el   6.51% abandonan sus estudios
En edad 3, 16   jovenes tienen 1 hijo(s), de los cuales el  75.00% abandonan sus estudios
En edad 3, 5    jovenes tienen 2 hijo(s), de los cuales el 100.00% abandonan sus estudios
En edad 3, 4    jovenes tienen 3 hijo(s), de los cuales el 100.00% abandonan sus estudios
Estudio no es independiente de cantidad de hijos

Para el cluster 1
En edad 3, 25933 jovenes tienen 0 hijo(s), de los cuales el   2.51% abandonan sus estudios
En edad 3, 17    jovenes tienen 1 hijo(s), de los cuales el  56.25% abandonan sus estudios
En edad 3, 2     jovenes tienen 2 hijo(s), de los cuales el 100.00% abandonan sus estudios
En edad 3, 3     jovenes tienen 3 hijo(s), de los cuales el 100.00% abandonan sus estudios
Estudio no es independiente de cantidad de hijos



## Escena 5:
Edad 4 (15 a 19) los jovenes que ya mantienen su hogar deberían ser alfabetas para poder trabajar y mantenerse dignamente, se tiene encuenta "parentesco", se espera que si tiene el valor "Jefe_Hogar" o "Pareja" entonces "alfabeta" sea "True". ¿Se filtra población?

In [7]:
for i, cabecera in enumerate(df['cabecera'].unique()):
    print('Para el cluster {0:d}'.format(i))
    # EDA
    df_slice = df[(df['edad'] == 4) & (df['cabecera'] == cabecera) & ~df['alfabeta']].copy()
    jefe_counts_h = df_slice[(df_slice['sexo'] == 'Hombre')]['parentesco'].value_counts()
    jefe_counts_m = df_slice[(df_slice['sexo'] == 'Mujer')]['parentesco'].value_counts()
    porcentaje_jefe_h = jefe_counts_h[['Jefe_Hogar', 'Pareja']].sum() / jefe_counts_h.sum()
    porcentaje_jefe_m = jefe_counts_m[['Jefe_Hogar', 'Pareja']].sum() / jefe_counts_m.sum()
    print('La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_h * 100))
    print('La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['parentesco'].isin(['Jefe_Hogar', 'Pareja']), columns = df_slice['sexo'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Manutención de hogar {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 12.00%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 28.46%
Manutención de hogar no es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 19.78%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 18.75%
Manutención de hogar no es independiente de sexo



## Escena 6
Edad 5 (20 a 25) luego de haber logrado el camino de la educación, de los no hijos y de la salud, ahora se evalúan las condiciones de vida. Empezamos por analizar cuántos educados no poseen remuneración.

In [8]:
crosstab_counts = pd.crosstab(index = df_slice['parentesco'].isin(['Jefe_Hogar', 'Pareja']), columns = [df_slice['sexo'], df_slice['nivel_educ']])
scipy.stats.chi2_contingency(crosstab_counts)

(2.899586009004107,
 0.4073675455317509,
 3,
 array([[36.93693694, 92.34234234, 22.16216216, 53.55855856],
        [ 3.06306306,  7.65765766,  1.83783784,  4.44144144]]))

In [9]:
for i, cabecera in enumerate(df['cabecera'].unique()):
    print('Para el cluster {0:d}'.format(i))
    # EDA
    df_slice = df[(df['edad'] == 5) & (df['cabecera'] == cabecera)].copy()
    jefe_counts_h = df_slice[(df_slice['sexo'] == 'Hombre')]['nivel_educ'].value_counts()
    jefe_counts_m = df_slice[(df_slice['sexo'] == 'Mujer')]['nivel_educ'].value_counts()
    porcentaje_jefe_h = jefe_counts_h[['Jefe_Hogar', 'Pareja']].sum() / jefe_counts_h.sum()
    porcentaje_jefe_m = jefe_counts_m[['Jefe_Hogar', 'Pareja']].sum() / jefe_counts_m.sum()
    print('La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_h * 100))
    print('La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['parentesco'].isin(['Jefe_Hogar', 'Pareja']), columns = df_slice['sexo'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Manutención de hogar {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 0


KeyError: 'a list-indexer must only include values that are in the categories'